In [1]:
import mPyPl as mp

In [2]:
%run features_extractor.py

Using TensorFlow backend.


In [3]:
from pipe import Pipe
import numpy as np

@Pipe
def apply_batch(datastream, src_field, dst_field, func, batch_size=32):
    """
    Apply function to the field in batches. `batch_size` elements are accumulated into the list, and `func` is called
    with this parameter.
    """
    n=0
    arg=[]
    seq=[]
    for x in datastream:
        f = __fextract(x,src_field)
        arg.append(f)
        seq.append(x)
        n+=1
        if (n>=batch_size):
            res = func(arg)
            for u,v in zip(seq,res):
                u[dst_field] = v
                yield u
            n=0
            arg=[]
            seq=[]
    if n>0: # flush remaining items
        res = func(arg)
        for u, v in zip(seq, res):
            u[dst_field] = v
            yield u
            

#### Functions below are copied from mPyPl core and will be eventually removed
def __fextract(x,field_name):
    """
    Extract value of a given field or fields.
    :param x: mdict
    :param field_name: name of a field or list of field names
    :return: value or list of values
    """
    if field_name is None: return x
    if isinstance(field_name, list) or isinstance(field_name, np.ndarray):
        return [x[key] for key in field_name]
    else:
        return x[field_name]


def __fnapply(x,src_field,func):
    """
    Internal. Apply function `func` on the values extracted from dict `x`. If `src_fields` is string, function of one argument is expected.
    If `src_fields` is a list, `func` should take list as an argument.
    """
    return func(__fextract(x,src_field))

In [ ]:
data_dir = "101_ObjectCategories"
data = (mp.get_datastream(data_dir, ext=".jpg")
 | mp.apply('filename', 'npy', lambda x: get_images([x], target_size=(200,200), color='RGB', bg_clr=0)[0])
 | apply_batch(
     src_field='npy',
     dst_field='emb',
     func=lambda x: extract_features(
         np.asarray(x),
         pretrained_model="resnet50",
         pooling_method='avg'),
     batch_size=64)
 | mp.as_list)

Input images shape:  (64, 200, 200, 3)


C:\Anaconda3\envs\py35\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
D:\git\images-vector-search\features_extractor.py:114: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  model = Model(input=base_model.input, output=[x])


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(data[0]['npy'])
